#### IBM Quantum Challenge Africa 2021より
# リョウコと量子コンピューターで農地収穫量最適化！

みなさん、「リョウコと量子コンピューターで農地収穫量最適化！」へようこそ。本ハンズオンでは、量子計算における問題のモデル化の例として、IBM Quantum Challenge Africa 2021で取り上げられた「農地収量の最適化問題」を解説します。ここで使用しているコードは、[Quantum Challenge Lab1のnotebook](https://github.com/qiskit-community/ibm-quantum-challenge-africa-2021/blob/main/content/lab1/lab1.ipynb)がベースになっています。

**前提知識**：Python

**事前準備**：[IBM Quantum](https://quantum-computing.ibm.com/)へのサインアップ


## 目次
1. [はじめに](#Introduction)
1. [Qiskit](#Qiskit)
1. [2次計画問題](#QuadraticProblem)
1. [農地の収穫量問題](#CropYieldProblem)
1. [Quadratic Unconstrained Binary Optimization (QUBO)](#QUBO)
1. [古典的な解法](#ClassicalSolution)
1. [量子的な解法](#QuantumSolution)
    1. [QAOAによる解法](#QAOASolution)
    1. [VQEによる解法](#VQESolution)
1. [まとめ](#Summary)
1. [参考文献](#Reference)

## はじめに <a id='Introduction'></a>

量子コンピューターは、古典コンピューターでは解けない問題を解決する可能性を秘めています。ただし全ての問題において古典コンピューターを凌駕するわけではありません。どのような問題が量子計算に適しているか、計算方法に適合するよう問題をどのようにモデル化するか、どのような入出力が適切なのか知ることが重要です。

実社会で有用な計算において量子コンピュータが古典コンピューターの計算速度を上回り、その優位性を示す領域を特定することは、量子コンピューターが実用化される時代に向けてとても重要なことです。このような優位性が期待される領域には、機械学習、計算化学、組み合わせ最適化問題などが挙げられます。今回は農地の収穫量を題材に、組み合わせ最適化問題を取り上げます。このハンズオンを通じ、量子コンピューターの優位性と今抱える問題について理解することができるでしょう。

まずは、ハンズオンを実行する環境を準備しましょう。

1. Jupyter notebookのダウンロード<br/>
  ハンズオンで使用するJupyter notebookファイルを[こちら](https://github.com/purepureclub/IFCO2021DEC)からダウンロードします

2. [IBM Quantum](https://quantum-computing.ibm.com/)へのログイン

3. ステップ1でダウンロードしたファイルのアップロード

  3-1. [Dashbord](https://quantum-computing.ibm.com/)上の[Launch Lab](https://quantum-computing.ibm.com/lab)ボタンをクリックします
  
  3-2. Upload Filesボタン(上矢印)を押して、ステップ1でダウンロードしたファイルをアップロードします <br/>

4. アップロードしたファイル「IFCO2021Dec_qiskit_handson.ipynb」を開いてください

次に、必要なモジュールとライブラリをインポートしておきましょう。Jupyter notebookでは、セルにカーソルを置き、Shift+Enterを押すと、セル内のコードが実行されます。

In [1]:
# 補助ライブラリのインポート
import numpy as np

# Qiskitのインポート
from qiskit import IBMQ, Aer
from qiskit.algorithms import QAOA, VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.providers.ibmq import least_busy
from qiskit.utils import QuantumInstance
from qiskit.tools import job_monitor

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo

以上で準備は完了です。

## Qiskit <a id='Qiskit'></a>

Qiskitは量子コンピューターを扱うためのオープンソースのフレームワークです。回路やパルス、アルゴリズムといった様々なレベルのライブラリが揃っています。Qiskitを使えば、量子回路を簡単に作成し、シミュレーターや実際の量子コンピュータを実行することができます。Qiskitは2つの基本ライブラリと、特定の応用的な問題を扱う4つのアプリケーション・モジュールから構成されます。

#### 基本ライブラリ
- Terra: 量子プログラムの基礎となる回路とパルス
- Aer: シミュレーター

#### アプリケーション・モジュール
- Qiskit Finance: ポートフォリオ最適化など金融分野
- Qiskit Machine Learning: SVM、ハイブリッド量子-古典ニューラルネットワークなど量子機械学習
- Qiskit Nature: 分子の基底エネルギーを求めるなど自然科学
- Qiskit Optimization: 最適化問題

これらのモジュールを使用すると、量子コンピューターの基礎知識がなくとも、特定の問題を量子コンピューターで解決するコードを作成することができます。モジュールの詳細について知りたい場合はチュートリアル([Finance](https://qiskit.org/documentation/finance/tutorials/index.html), [Machine Learning](https://qiskit.org/documentation/machine-learning/locale/ja_JP/tutorials/index.html), [Nature](https://qiskit.org/documentation/nature/tutorials/index.html), [Optimization](https://qiskit.org/documentation/optimization/tutorials/index.html))を参照するとよいでしょう。本ハンズオンでは、`Qiskit Optimization`モジュールを使用して、量子コンピューターを使った最適化問題の解法について解説します。

#### Qiskit Optimization
Qiskit Optimizationはアプリケーション・モジュールの内の一つで最適化問題に特化したモジュールです。Qiskit optimizationは最適化問題を量子回路に変換し、量子回路の実行結果（通常、ビット列）を元の問題の変数の割当に再変換します。

- 最適化問題を解くプロセスのすべてをカバー
- 最適化問題のハイレベルなモデル作成
- 最適化問題を自動で量子コンピュータ用の形式（イジングハミルトニアンに変換）
- 問題を解くための豊富なアルゴリズムライブラリー
  - QAOA, Grover adaptive Search, Classical solvers


#### 最適化問題
- 様々な分野の実用的な問題の多くは最適化問題であり、複雑な意思決定や戦略の定義の核となります
- 最適化問題とは、有限または可算的に無限に存在する潜在的な解の集合の中から、与えられた制約条件のもとで、最適な解を探し出す問題です
- 最適性は、最大化(もしくは最小化)される目的関数によって定義されます

#### 代表的な目的関数
- 最小化：コスト、距離、重量、処理時間
  - 例）AからBへ最短時間で行く経路を求める
- 最大化：利益、価値、生産高、効率、満足度
  - 例）300円の予算内で最も満足度が高い遠足に持っていくお菓子を決める

## 2次計画問題 <a id='QuadraticProblem'></a>
制約条件と目的関数を変数の2次式として数式化できる最適化問題を、2次計画問題と呼びます。このような問題は、金融、農業、運用・生産管理、経済学などの分野で見ることができますが、このハンズオンのベースとなったQuantum Challenge Africa 2021では、アフリカ大陸が抱える問題に応用するという観点から、農業に焦点を当てました。

2次計画問題の厳密な定義は以下の通りです。

$n$次元のベクトル変数 $x\in\mathbb{R}^n$ が与えられたとき：

$$
\begin{align}
\text{minimize or maximize}\quad & f\left(x\right)=\frac{1}{2}x^\top{}\mathbf{Q}x + c^\top{}x &\\
\text{subject to}\quad & \mathbf{A}x\leq{}b&\\
& x^\top{}\mathbf{Q}_ix + c_{i}^\top{}x\leq{}r_i,\quad&\forall{}i\in[1,k_q]\\
& l_i\leq{}x_i\leq{}u_i,\quad&\forall{}i\in[1,k_l]\\
\end{align}
$$

$\mathbf{Q}$、$\mathbf{Q}_i$ 、$\mathbf{A}$ は $n\times{}n$ の対称行列、$c$ と $c_i$ は $n\times{}1$ の列ベクトル。 $\mathbf{Q}_i$、$\mathbf{A}$、 $c_i$、 $l_i$、 $u_i$ は、変数 $x$ の制約を定義。制約式にはあらゆる不等号('$<$', '$=$', '$>$', '$\geq$', '$\leq$')が使用可。


3つの整数変数を持つ例：

$$\begin{align}
\text{minimize}\quad{} & f(x)=x_1^2 + x_2^2 - x_1x_2 - 6x_3 \\
\text{subject to}\quad{} & x_1 + x_2 + 2x_3 \leq{} 5             \\
                         & 0 \leq{} x_1 \leq{} 4    \\
                         & -2 \leq{} x_2 \leq{} 2    \\
                         & -2 \leq{} x_3 \leq{} 4    \\
\end{align}$$

簡単に言うと、変数の2次式で表される目的関数を、2次式で表せる制約条件において、最小化もしくは最大化する変数を見つける問題です。

Qiskitにおいて、二次計画問題を扱うには、問題を `QuadraticProgram` インスタンスとして定義する必要があります。上記の例を`QuadraticProgram` インスタンスとして定義してみましょう。インターフェースの詳細は、Qiskitドキュメンテーションの[`QuadraticProgram`](https://qiskit.org/documentation/stubs/qiskit.optimization.QuadraticProgram.html)クラスを参照してください。

最小化する2次式は、dictionaryを使って実装されます。dictionaryのキーは、$f(x)$の項を表す変数名です。例えば、`("x_1", "x_2")`は、$x_1x_2$に対するものです。dictionaryの値は、その項の係数です。減算される項は、負の係数になります。

In [ ]:
# インスタンスの作成
exprog = QuadraticProgram(name="example")
# 変数の追加
exprog.integer_var(name="x_1", lowerbound=0, upperbound=4)
exprog.integer_var(name="x_2", lowerbound=-2, upperbound=2)
exprog.integer_var(name="x_3", lowerbound=-2, upperbound=4)
# 目的関数の定義
exprog.minimize(
    linear={"x_3": -6},
    quadratic={("x_1", "x_1"): 1, ("x_2", "x_2"): 1, ("x_1", "x_2"): -1},
)
# 制約の追加
exprog.linear_constraint(linear={"x_1": 1, "x_2": 1, "x_3": 2}, sense="<=", rhs=5)

例題再掲：
$$\begin{align}
\text{minimize}\quad{} & f(x)=x_1^2 + x_2^2 - x_1x_2 - 6x_3 \\
\text{subject to}\quad{} & x_1 + x_2 + 2x_3 \leq{} 5             \\
                         & 0 \leq{} x_1 \leq{} 4    \\
                         & -2 \leq{} x_2 \leq{} 2    \\
                         & -2 \leq{} x_3 \leq{} 4    \\
\end{align}$$

`QuadraticProgram`は、バイナリ、整数、連続の3種類の変数を持つことができます。現在Qiskitでは、これから使用するアルゴリズムにおいてバイナリ変数と整数変数のみをサポートしています。連続変数のシミュレーションを可能にするアルゴリズムについて詳しく知りたい方は、Qiskitチュートリアルの[algorithm to solve mixed-variable quadratic problems](https://qiskit.org/documentation/tutorials/optimization/5_admm_optimizer.html)を見てください。

`QuadraticProgram`のインスタンスは、LP文字列(**L**inear **P**rogramming 文字列)として可視化できます。

In [ ]:
print(exprog.export_as_lp_string())

2次計画問題は、一般的にNP困難と呼ばれ、古典的な計算方法では解くことが難しい問題群です。特定の場合に限り古典的な手法を用いて厳密解を導くことができますが、一般的に古典的な解法として、近似解を求めるヒューリスティック・アルゴリズムがよく用いられます。量子コンピューターは、ヒューリスティック・アルゴリズムを用いた解法において、大幅なスピードアップと優れたスケーリングを実現することが期待されています。つまり、2次計画問題は量子コンピューターの優位性が期待されるエリアなのです。

## 農地の収穫量問題 <a id="CropYieldProblem"></a>
農場の作物や経営を最適化して、リスクを減らしながら利益を上げたいというニーズはよくあります。アフリカや全世界が直面している大きな課題の1つは、すべての人に十分な食料をいかに生産するかということです。今回問題とするのは、利益ではなく、収穫した作物のトン数です。農地の広さが決められ、植えられる作物の種類が与えられたとき、どの作物をそれぞれ何ヘクタール植えたら、収穫量を最大にできるかが課題になります。

農法には「単作」「間作」「プッシュプル農法」の3種類があります。単作とは、一つの作物だけを栽培する方法です。単作では、病気や害虫の影響を受けやすく、収穫量全体に影響を及ぼします。また、異なる作物を近くで栽培すると、両方の収穫量が増えたり、逆に収穫量が減ったりします。間作とは、収穫量を _増やす_ ために2つの異なる作物を選択することです。プッシュプル農法とは、害虫を寄せ付けないプッシュプル作物と、害虫を引き寄せる作物をペアで栽培することです。これを大規模な農場に組み込むことで、収穫量を増やすことができます。ただ、プッシュプル作物は利用できなかったり食用にならないため、プッシュプル作物を全体の収穫量として使用できません。

このような農地の収穫量問題は、変数の特定の組み合わせで解が得られるという点で、組合せ最適化問題です。ここで紹介する問題は古典的に解くことができるほど小さいものですが、より大きな問題になると、最適化すべき組み合わせの数が多くなるため、古典的なコンピュータでは扱いにくくなります。

### 問題の定義
あなたは$3~ha$の農場のオーナーです。植えられる作物は、小麦、大豆、トウモロコシ、プッシュプルの4種類です。プッシュプルは、収穫しても売ることはできませんが、他の作物の収穫量を増やすことができます。それぞれの作物は$0~ha$または$1~ha$作付けすることができます。この農場の収穫量(トン)を以下のような2次関数として定義します。この2次関数の変数は、作付けする作物のヘクタール数です。このシナリオでは、すべての作物が他の作物の収穫量を増加させます。解決すべき問題は、どの作物を使えば最大の収穫量が得られるか(2次関数を最大化できるか)ということです。

$$
\begin{align}
    \text{maximize} \quad & 2(\operatorname{Wheat}) + \operatorname{Soybeans} + 4(\operatorname{Maize}) \\
    & + 2.4(\operatorname{Wheat}\times\operatorname{Soybeans}) \\ & + 4(\operatorname{Wheat}\times\operatorname{Maize})\\
    &+ 4(\operatorname{Wheat}\times\operatorname{PushPull}) \\ & + 2(\operatorname{Soybeans}\times\operatorname{Maize}) \\
                          & + (\operatorname{Soybeans}\times\operatorname{PushPull}) \\ & + 5(\operatorname{Maize}\times\operatorname{PushPull})
\end{align}
$$

$$
\begin{align}
\text{subject to} \quad & \operatorname{Wheat} + \operatorname{Soybeans} + \operatorname{Maize} + \operatorname{PushPull} \leq{} 3\\
& 0\leq{}\operatorname{Wheat}\leq{}1\\
& 0\leq{}\operatorname{Soybeans}\leq{}1\\
& 0\leq{}\operatorname{Maize}\leq{}1\\
& 0\leq{}\operatorname{PushPull}\leq{}1
\end{align}
$$

### 作物収量変数から2次計画問題の作成
上記のモデルを表す`QuadraticProgram`を作成してみましょう。以下の `cropyield_quadratic_program` 関数を完成させてください。先程の例を参考に、かつ [QuadraticProgram ドキュメンテーション](https://qiskit.org/documentation/tutorials/optimization/1_quadratic_program.html?highlight=quadraticprogram) と [Qiskitリファレンス](https://qiskit.org/documentation/stubs/qiskit.optimization.QuadraticProgram.html?highlight=quadraticprogram#qiskit.optimization.QuadraticProgram) を参照するとよいでしょう。

変数名は何でもよいですが、 `Wheat`, `Soybeans`, `Maize`, および `PushPull` の頭文字を使うとわかりやすいです。

In [ ]:
def cropyield_quadratic_program():
    cropyield = QuadraticProgram(name="Crop Yield")
    ##############################
    # ここにコードを書いてください
    cropyield.binary_var(name="W")

    
    cropyield.maximize(
        linear={ },
        quadratic={ },
    )
    cropyield.linear_constraint(linear={ }, sense="", rhs=)

    #
    ##############################
    return cropyield

# 問題をLP文字列で出力
cropyield = cropyield_quadratic_program()
print(cropyield.export_as_lp_string())

## Quadratic Unconstrained Binary Optimization (QUBO) 形式への変換 <a id="QUBO"></a>


量子コンピューターで2次計画問題を解く際に広く使われているのが、Quadratic Unconstrained Binary Optimization (QUBO) 形式へ変換する手法です。QUBOとは、その名の通り、制約のないバイナリ変数を用いた2次の最適化問題です。2次計画問題をQUBO形式に変換することで、量子コンピューターで計算可能なイジングモデルのハミルトニアンの基底状態を求める問題に帰着することができます。



<!-- イジングモデルとは、磁性体を量子的な磁石スピンの集合体として表現したモデルで、各スピンは上向きと下向きの重ね合わせ状態を取り、スピンどうしの相互作用により安定な状態（基底状態）へ移行します。スピンの上向き・下向きを、バイナリ変数の0、1に読み替えることで、QUBO問題をイジングモデルにマップするのです。

<img src="ising_model.svg" width="400px"/> -->

Qiskitは、2次計画問題をQUBOに変換する `QuadraticProgramToQUBO` クラスを提供しています。内部的な動きは以下の通りです。
1. 特殊な不等制約式を目的関数のペナルティ項に変換
1. **不等式制約**を等式制約に変換する <- **変数の追加**
1. **整数変数**をバイナリ変数に変換 <- **変数の追加**
1. 等式制約を目的関数のペナルティ項に変換

変数の増加は、量子コンピューター上で必要になる量子ビット数の増加を意味します。つまり、一般的な不等式制約や多くの整数変数がある2次計画問題は、多くの量子ビットを必要とするのです。例えば、最初に作った2次計画問題の例をみてみます。

In [ ]:
print(exprog.export_as_lp_string())

このように例題は3つの変数で定義されたとてもシンプルな問題です。ですが、QUBOに変換すると13の変数が必要になり、目的関数もとても複雑になります。実量子コンピューターがノイズあり中規模量子デバイスである現在、量子ビット数を少なくすることは重要です。使用する量子ビット数が少なくなるような問題のモデル化が鍵と言えます。

In [ ]:
QuadraticProgramToQubo().convert(exprog)

私たちの農地収量問題をQUBOに変換しましょう。変数は全てバイナリ変数ですが、一般的な不等式制約がありますので、QUBOにすると、変数とペナルティ項が追加されます。

In [ ]:
##############################
# ここにコードを書いてください。変数はいくつになりますか？
cropyieldQUBO=
#
##############################
print(cropyieldQUBO.export_as_lp_string())

## 古典的な解法 <a id="ClassicalSolution"></a>

以上で、農地収量最適化問題を量子コンピューターで解ける形に定義することができました。この問題はさほど大きくないので、古典的に厳密解を求めることも可能です。実際に量子コンピューターで解く前に、古典的に解を求めておきましょう。

古典的な解は、NumpyとQiskitを使って簡単に得ることができます。QUBO問題を解くことは、基礎となる行列表現の最小固有値を見つけることに他なりません。そして幸いなことに、この行列表現がどのようなものかを知る必要はなく、定義したQUBO問題を、`MinimumEigensolver`と`MinimumEigenOptimizer`に渡すだけでよいのです。

古典解を求める関数は以下のように定義できます。まずソルバーとして`NumPyMinimumEigensolver`インスタンスを持つ、`MinimumEigenOptimizer` オプティマイザーを作成します。与えられた2次計画問題を引数として`solve` メソッドを呼ぶと、オプティマイザーは，問題をパラメーター化された表現に変換してソルバーに渡し、ソルバーはパラメーターを最適化することで、行列表現の最小固有値を求め、最終的に元の問題の解を得ることができます。

In [ ]:
def get_classical_solution_for(quadprog: QuadraticProgram):
    # Create solver
    solver = NumPyMinimumEigensolver()

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Return result from optimizer
    return optimizer.solve(quadprog)

私たちの農地収量問題を解いてみましょう。

In [ ]:
# Get classical result
classical_result = get_classical_solution_for(cropyield)

# Format and print result
print("Solution found using the classical method:\n")
print(f"Maximum crop-yield is {classical_result.fval} tons")
print(f"Crops used are: ")

_crops = [v.name for v in cropyield.variables]
for cropIndex, cropHectares in enumerate(classical_result.x):
    print(f"\t{cropHectares} ha of {_crops[cropIndex]}")

## 量子的な解法 <a id="QuantumSolution"></a>

では、量子コンピューターを使用して、農地収量問題を解いてみましょう。量子コンピューターでQUBOを解く方法はいくつかあります。今回は、[_Quantum Approximate Optimization Algorithm_](https://qiskit.org/documentation/stubs/qiskit.algorithms.QAOA.html?highlight=qaoa#qiskit.algorithms.QAOA) (QAOA)を使います。このファイルの最後に[_Variational Quantum Eigensolver_](https://qiskit.org/documentation/stubs/qiskit.algorithms.VQE.html?highlight=vqe#qiskit.algorithms.VQE) (VQE)を使用した解法も添付しましたので、興味のある方はハンズオン終了後にご覧ください。

QAOA、VQE共に、量子計算と古典計算を組み合わせたハイブリッド型のアルゴリズムで、量子計算で使用するパラメーターを変更するために古典的なオプティマイザーを使用します。どちらも、最適化するシステムを表す行列の最小固有値を見つけるもので、非常に人気のあるアルゴリズムです。

### QAOAによる解法 <a id="QAOASolution"></a>
QAOAを使って問題を解くには、先ほど定義した関数で、古典的ソルバーを`QAOA`クラスのインスタンスで置き換えるだけです。量子アルゴリズムを実行しているので、量子コンポーネントをどのバックエンドで実行するかをソルバーに伝える必要があります。QAOAは反復アルゴリズムなので、内部パラメータを変えて複数回実行します。このパラメータは、計算の最適化ステップにおいて、`optimizer`によって古典的に調整されます。`optimizer`を`None`にしておくと、アルゴリズムはデフォルトの最適化アルゴリズムを使用します。反復回数を決定するために、コールバック関数を定義しています。この関数は反復ごとに実行され、これまでの評価回数を格納します．アルゴリズムの実行終了時には、結果と反復回数を返します。

In [ ]:
def get_QAOA_solution_for(
    quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver
    solver = QAOA(
        optimizer=optimizer, quantum_instance=quantumInstance, callback=callback,
    )

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

#### シミュレーターでの実行
量子コンピューターでの実行準備ができました。まずはシミュレーターで実行してみましょう。QiskitのAerコンポーネントの`qasm_simpulator`を使用して、`QuantumInstance`を作成し、`get_QAOA_solution_for` 関数に渡します。

In [ ]:
# We will use the Aer provided QASM simulator
backend = Aer.get_backend("qasm_simulator")

# Create a QuantumInstance
simulator_instance = QuantumInstance(backend=backend)

# Get QAOA result
qaoa_result, qaoa_eval_count= get_QAOA_solution_for(cropyield, simulator_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum crop-yield is {qaoa_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

#### 実量子ハードウェア上での実行
次に実量子ハードウェアで実行したいところですが、実は私たちはこの量子回路を実量子ハードウェア上で動作させることができません。先ほど見たように私たちの問題を解くためには、6量子ビットが必要です。今私たちが使用できる量子コンピューターの量子ビット数を確認しましょう。

まず、IBM Quantumのアカウントをロードします。次のセルがエラーになる方は、この[クイックガイド](https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)に従って、アカウントを有効にしてください。

In [ ]:
provider = IBMQ.load_account()

次に、使用できるハードウェアの名前とビット数を確認します。

In [ ]:
for _backend in IBMQ.get_provider(hub='ibm-q', group='open', project='main').backends():
    if not (_backend.configuration().simulator):
        print(f"{_backend.name()} : {_backend.configuration().n_qubits} qubit")

このように、一般的な人は5量子ビットの量子コンピューターしか使えません。6量子ビット必要な私たちの問題は実行できないのです。

ですが、せっかくですので、問題のサイズを小さくして実量子ハードウェア上で実行しましょう。$4~ha$の土地に、小麦かトウモロコシを$2~ha$以下ずつ植え付ける問題に変更します。これは、単一の作物を栽培しすぎた場合の影響を示すためのモデルです。単一の作物を栽培するヘクタール数を増やすと収量が減少します。しかし、小麦とトウモロコシの両方を使用すると収量が増加し、間作の利点を示しています。また，最大で $4~ha$ まで使用できますが，モデルがこの限界を超えることはないため，最大ヘクタール数を定義する線形制約は含めていません．これにより、必要な量子ビットの数は6から4に減ります。

In [ ]:
# Create a small crop-yield example quadratic program
cropyield_small = QuadraticProgram(name="Small Crop-Yield")

# Add two variables, indicating whether we grow 0, 1, or 2 hectares for two different crops
cropyield_small.integer_var(lowerbound=0, upperbound=2, name="Wheat")
cropyield_small.integer_var(lowerbound=0, upperbound=2, name="Maize")

# Add the objective function defining the yield in tonnes
cropyield_small.maximize(
    linear={"Wheat": 3, "Maize": 3},
    quadratic={("Maize", "Wheat"): 1, ("Maize", "Maize"): -2, ("Wheat", "Wheat"): -2},
)

# This linear constraint is not used as the model never reaches this. This is because the
# sum of the upperbounds on both variables is 4 already. If this constraint is applied, the
# model would require 6 qubits instead of 4.
# cropyield_small.linear_constraint(linear={"Wheat": 1, "Maize": 1}, sense="<=", rhs=4)

print(cropyield_small)

QUBOに変換して必要な量子ビット数を確認しましょう。

In [ ]:
QuadraticProgramToQubo().convert(cropyield_small)

4量子ビットで実行可能であることがわかったので、実行できる量子コンピューターを探します。

In [ ]:
least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 4 
                             and not x.configuration().simulator))

見つけた量子コンピューター上で、QAOAアルゴリズムを実行してみましょう。実機で実行する前に、先ほどと同じく、古典解とシミュレーター解も取得しておきます。また、長く量子コンピューターを占有しないよう、オプティマイザーと最大反復数を指定しておきます。コードは以下の通りですが、今みなさんで実機で実行するとキューにたくさん詰まってしまい、とても時間がかかってしまいますので、ハンズオン終了後に実行してください。

In [ ]:
# Get classical result
classical_result = get_classical_solution_for(cropyield_small)

# Format and print result
print("Solution found using the classical method:\n")
print(f"Maximum crop-yield is {classical_result.fval} tons")
print(f"Crops used are: ")

_crops = [v.name for v in cropyield.variables]
for cropIndex, cropHectares in enumerate(classical_result.x):
    print(f"\t{cropHectares} ha of {_crops[cropIndex]}")

In [ ]:
# Get simulated result
backend = Aer.get_backend("qasm_simulator")

# Create a QuantumInstance
simulator_instance = QuantumInstance(backend=backend)

# Create our optimizer
optimizer = COBYLA(maxiter=50)

# Get QAOA result
qaoa_result, qaoa_eval_count= get_QAOA_solution_for(cropyield_small, simulator_instance, optimizer=optimizer)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum crop-yield is {qaoa_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

In [ ]:
# Get result from real quantum computing
backend_real = provider.get_backend("ibmq_belem")
quantum_instance_real = QuantumInstance(backend_real, shots=2048)

# Create our optimizer
optimizer = COBYLA(maxiter=50)

## Get result from real device with VQE
qaoa_result_real, qaoa_eval_count_real = get_QAOA_solution_for(
    cropyield_small, quantum_instance_real, optimizer=optimizer
)
# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum crop-yield is {qaoa_result_real.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(qaoa_result_real.x, qaoa_result_real.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

### (オプション) VQEによる解法 <a id="VQESolution"></a>

QUBO問題を解くには、`VQE` アルゴリズムを使用することもできます。使用方法は以下の通りです。`VQE` のインスタンスも反復処理を行いますので、作物収量問題の解決策を見つけるために何回の反復処理が必要かを測定することができます。

In [ ]:
def get_VQE_solution_for(
    quadprog: QuadraticProgram, quantumInstance: QuantumInstance, optimizer=None,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver and optimizer
    solver = VQE(
        optimizer=optimizer, quantum_instance=quantumInstance, callback=callback
    )

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

シミュレーターに対して実行してみてください。以前とほぼ同じ答えが返ってくるはずです(乱数のシードを固定していないので多少の変動があります)。

In [ ]:
# We will use the Aer provided QASM simulator
backend = Aer.get_backend("qasm_simulator")

# Create a QuantumInstance
simulator_instance = QuantumInstance(backend=backend)

# Get VQE result
vqe_result, vqe_eval_count = get_VQE_solution_for(cropyield, simulator_instance)

# Format and print result
print("Solution found using the VQE method:\n")
print(f"Maximum crop-yield is {vqe_result.fval} tons")
print(f"Crops used are: ")
for cropHectares, cropName in zip(vqe_result.x, vqe_result.variable_names):
    print(f"\t{cropHectares} ha of {cropName}")

print(f"\nThe solution was found within {vqe_eval_count} evaluations of VQE")

## まとめ <a id="Summary"></a>

いかがでしたか？本日はハンズオンを通して以下を説明しました：

- Qiskit Optimizationを用いた2次計画問題のプログラミング方法
- 作成したプログラムの古典的＆量子的実行方法

量子コンピューターが最適化問題の解法として優越性が期待されているものの、実際の問題を解くためには、量子ビット数を削減できるようなモデルに落とし込むことが重要であることがお分かりいただけましたでしょうか。

この問題が取り上げられたQuantum Challenge 2021 Africaの問題は[こちら](https://github.com/qiskit-community/ibm-quantum-challenge-africa-2021)から取得可能です。このハンズオンでご紹介した問題のほか、金融や創薬の問題があります。また先日終了した[Quantum Challenge 2021 Fall](https://github.com/qiskit-community/ibm-quantum-challenge-fall-2021)にも興味深い問題がたくさんあります。こちらは日本語訳もありますので、より取り組みやすいと思います。来年もChallengeがあると予想されます。規定数の問題を解くとバッジが取得できますので、みなさまもぜひ挑戦してみてください！

また、Qiskitをもっと知りたい方には、Qiskit Documentationの[Tutorial](https://qiskit.org/documentation/tutorials.html)を、量子コンピューターについてもっと勉強したい方には、[Qiskit Textbook](https://qiskit.org/textbook/preface.html)をお勧めします。TutorialはIBM Quantumの[Quantum Lab](https://quantum-computing.ibm.com/lab)からも参照＆実行できます。

## 参考文献 <a id="Reference"></a>
[1] A. A. Nel, ‘Crop rotation in the summer rainfall area of South Africa’, South African Journal of Plant and Soil, vol. 22, no. 4, pp. 274–278, Jan. 2005, doi: 10.1080/02571862.2005.10634721.

[2] H. Ritchie and M. Roser, ‘Crop yields’, Our World in Data, 2013, [Online]. Available: https://ourworldindata.org/crop-yields.

[3] G. Brion, ‘Controlling Pests with Plants: The power of intercropping’, UVM Food Feed, Jan. 09, 2014. https://learn.uvm.edu/foodsystemsblog/2014/01/09/controlling-pests-with-plants-the-power-of-intercropping/ (accessed Feb. 15, 2021).

[4] N. O. Ogot, J. O. Pittchar, C. A. O. Midega, and Z. R. Khan, ‘Attributes of push-pull technology in enhancing food and nutrition security’, African Journal of Agriculture and Food Security, vol. 6, pp. 229–242, Mar. 2018.

In [1]:
import qiskit.tools.jupyter

#%qiskit_version_table
%qiskit_copyright

ワークショップ、セッション、および資料は、IBMまたはセッション発表者によって準備され、それぞれ独自の見解を反映したものです。それらは情報提供の目的のみで提供されており、いかなる参加者に対しても法律的またはその他の指導や助言を意図したものではなく、またそのような結果を生むものでもありません。本講演資料に含まれている情報については、完全性と正確性を期するよう努力しましたが、「現状のまま」提供され、明示または暗示にかかわらずいかなる保証も伴わないものとします。本講演資料またはその他の資料の使用によって、あるいはその他の関連によって、いかなる損害が生じた場合も、IBMは責任を負わないものとします。本講演資料に含まれている内容は、IBMまたはそのサプライヤーやライセンス交付者からいかなる保証または表明を引きだすことを意図したものでも、IBMソフトウェアの使用を規定する適用ライセンス契約の条項を変更することを意図したものでもなく、またそのような結果を生むものでもありません。

本講演資料でIBM製品、プログラム、またはサービスに言及していても、IBMが営業活動を行っているすべての国でそれらが使用可能であることを暗示するものではありません。本講演資料で言及している製品リリース日付や製品機能は、市場機会またはその他の要因に基づいてIBM独自の決定権をもっていつでも変更できるものとし、いかなる方法においても将来の製品または機能が使用可能になると確約することを意図したものではありません。本講演資料に含まれている内容は、参加者が開始する活動によって特定の販売、売上高の向上、またはその他の結果が生じると述べる、または暗示することを意図したものでも、またそのような結果を生むものでもありません。パフォーマンスは、管理された環境において標準的なIBMベンチマークを使用した測定と予測に基づいています。ユーザーが経験する実際のスループットやパフォーマンスは、ユーザーのジョブ・ストリームにおけるマルチプログラミングの量、入出力構成、ストレージ構成、および処理されるワークロードなどの考慮事項を含む、数多くの要因に応じて変化します。したがって、個々のユーザーがここで述べられているものと同様の結果を得られると確約するものではありません。

記述されているすべてのお客様事例は、それらのお客様がどのようにIBM製品を使用したか、またそれらのお客様が達成した結果の実例として示されたものです。実際の環境コストおよびパフォーマンス特性は、お客様ごとに異なる場合があります。

IBM、IBM ロゴ、ibm.com、Qiskitは、世界の多くの国で登録されたInternational  Business  Machines  Corporationの商標です。他の製品名およびサービス名等は、それぞれIBMまたは各社の商標である場合があります。現時点でのIBM の商標リストについては、www.ibm.com/legal/copytrade.shtml をご覧ください。

JupyterはNumFOCUS foundationの登録商標です。

PythonはPython Software Foundationの登録商標です。